<img src="../images/AzPTravel_PPM.png">

## GPM Pipeline Output maker

#### This script
- loads the transformed parquet file
- creates the file {parquet}.localcur.csv
- makes the Euro conversions
- outputs {parquet}.euroconv.csv


#### Parameters section

In [ ]:
commit = ""
# give a reason for the run

percode = "20XX.QX"
# Data Collection Code, this controls file paths and output names

run_type = 1
#run_type =  0 - lite run with no reporting, not recommended.
#run_type =  1 - lite run with normal reporting, default setting.
#run_type =  2 - Heavy run with full reporting, available for audits and troubleshooting.

specialchars = "-GTHtest"
# optional - add up to a 12 character code in order to mark your instance record .ipynb

from datetime import datetime
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")
# a single datetime stamp for the full instance run


In [ ]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Demo Test"
inst_datetime = "05262021182323"


### Set Run Control

##### 0 - lite run with no reporting, not recommended.
##### 1 - lite run with normal reporting, default setting.
##### 2 - Heavy run with full reporting, available for audits and troubleshooting.

In [ ]:
run_control = 0 # in development mode

#### Packages used

In [ ]:
import sys
import os
import pandas as pd

In [ ]:
default_DC = "2021.Q1"

try:
    if sys.argv[1] == "-f":
        percode = default_DC
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_DC
except NameError:
    percode = default_DC

#### Convert paths and files to variables.
#### Read data into pd DataFrames.
#### Make paths for the live sources.

In [ ]:
rt_path = f'//hecate/Insurance_US/Product Development/Product Manag' \
          f'ement/Global PPM/Reporting/Data Collection/Production/{str(percode)}'
tempfile = os.path.join(rt_path, f'{str(percode)}.localcur.parquet')
gppm_file = os.path.join(rt_path, f'GPPM_Input_{str(percode)}.xlsx')

#### Read source

In [ ]:
mo_input = pd.read_parquet(tempfile, engine='pyarrow')

print(mo_input)

#### Output local currency csv

In [ ]:
mo_input.to_csv(f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}\\{str(percode)}.localcur.csv', ',', index=False, encoding='utf-8-sig')

#### Convert all currency to Euro


In [ ]:

cflds = ['written_revenues_net_of_taxes', 'written_revenues', 'earned_revenues_net_of_taxes',
         'earned_revenues', 'earned_base_commissions', 'earned_over_commissions', 'upfront_cash_payments',
         'total_compensation', 'paid_claims', 'ocr__ibnr', 'actual_incurred_losses',
         'internal_variable_costs', 'az_tech_fee', 'internal_fixed_costs',
         'hq_fees', 'total_expenses', 'risk_premium', 'profit_or_loss', 'cm_hq_view', 'cm_bu_view', ]


"""reads in the input for the Euro conversion"""
euro = pd.read_excel(gppm_file, sheet_name="Euro_Rates")


#### Make a copy for debug purposes

In [ ]:
trandata = mo_input.copy()

#### Convert Currency Below

In [ ]:
# create DataFrame of unique currencies with rates

fx_uni = euro[['CURR_CODE','RATE']]
fx_uni = fx_uni.drop_duplicates()

In [ ]:
# add euro conversion rates to trandata DataFrame as column 'eur_rate'

eurdata = pd.merge(trandata['currency'], fx_uni, left_on='currency', right_on='CURR_CODE', how='left')

trandata['eur_rate'] = eurdata['RATE']

#### Convert currency fields

In [ ]:
trandata[cflds] = trandata[cflds].divide(trandata['eur_rate'], axis="index").fillna(0)


#### Output to {percode}.euroconv.csv

In [ ]:
trandata.to_csv(f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}\\{str(percode)}.euroconv.csv', ',', index=False, encoding='utf-8-sig')




#### Store the DataFrame for other notebooks to use

In [ ]:
mo_df = trandata

%store mo_df
